In [1]:
import pandas as pd
pd.options.display.max_columns = 100

In [2]:
# FUNC_ANNOT = "pathway"
# FUNC_ANNOT = "COG"
FUNC_ANNOT = "regulon"

In [3]:
muts_df = pd.read_pickle("./data/glycerol_2_5_df.pkl")
display(len(muts_df))

107

In [4]:
set(muts_df.exp)

{'GYD', 'SSW_GLY'}

In [5]:
# processing to help a cleaner order of features.
# sorts mutations by position so that mutations to proximal features will be inserted into flow_df next to each other.
muts_df = muts_df.sort_values(by="Position")
muts_df.head()

,Details,Mut ID,Mutation Type,Position,RBS,Reference Seq,RegulonDB Gene,Sequence Change,TFBS,TSS,ale,attenuator terminator,base-media,carbon-source,cis-regulatory RNA,coding,disrupt,exp,flask,gene,gene count,genetic,genetic features,genomic features,index,isolate,mutation size,mutation target annotation,nitrogen-source,oriC,phosphorous-source,presence,promoter,pseudogene,range,repaired,strain-description,sulfur-source,supplement,taxonomy-id,tech_rep,temperature,terminator,genetic feature links,operons,operon links,COGs,COG links,regulators,regulator links,pathways,pathway links
1737,intergenic (‑58/+35),317489.0,SNP,83587.0,{},NC_000913,leuA/leuL,A→C,{},{},10.0,{ECK125143560},M9,glycerol(2.5),{ECK120001253},0.0,0.0,GYD,106.0,{},1.0,0.0,"[{'RegulonDB ID': 'ECK120001204/ECK120001253',...","[{'name': 'leuLABCD attenuator terminator', 'R...",27,0.0,1.0,leuA/leuL,NH4Cl(1),0.0,KH2PO4(3),0.59,{},0.0,"(83587, 83587)",0.0,BOP27,MgSO4(0.24),trace elements,511145,1.0,37 celsius,{ECK120034959},{'ECK120001204/ECK120001253': ['ECK125143560']},"[{'name': 'leuLABCD', 'RegulonDB ID': 'ECK1200...",{'ECK120014726': ['ECK120001204/ECK120001253']},[{'name': 'Amino acid transport and metabolism...,{'Amino acid transport and metabolism': ['ECK1...,"[{'name': 'LeuO', 'mutation set count': 3.0, '...",{'LeuO': ['leuLABCD']},"[{'name': 'Valine, leucine and isoleucine bios...","{'Valine, leucine and isoleucine biosynthesis'..."
1763,intergenic (‑58/+35),317489.0,SNP,83587.0,{},NC_000913,leuA/leuL,A→C,{},{},14.0,{ECK125143560},M9,glycerol(2.5),{ECK120001253},0.0,0.0,GYD,68.0,{},1.0,0.0,"[{'RegulonDB ID': 'ECK120001204/ECK120001253',...","[{'name': 'leuLABCD attenuator terminator', 'R...",27,0.0,1.0,leuA/leuL,NH4Cl(1),0.0,KH2PO4(3),0.50,{},0.0,"(83587, 83587)",0.0,BOP27,MgSO4(0.24),trace elements,511145,1.0,37 celsius,{ECK120034959},{'ECK120001204/ECK120001253': ['ECK125143560']},"[{'name': 'leuLABCD', 'RegulonDB ID': 'ECK1200...",{'ECK120014726': ['ECK120001204/ECK120001253']},[{'name': 'Amino acid transport and metabolism...,{'Amino acid transport and metabolism': ['ECK1...,"[{'name': 'LeuO', 'mutation set count': 3.0, '...",{'LeuO': ['leuLABCD']},"[{'name': 'Valine, leucine and isoleucine bios...","{'Valine, leucine and isoleucine biosynthesis'..."
1738,intergenic (‑61/+32),317490.0,SNP,83590.0,{},NC_000913,leuA/leuL,G→T,{},{},10.0,{ECK125143560},M9,glycerol(2.5),{ECK120001253},0.0,0.0,GYD,106.0,{},1.0,0.0,"[{'RegulonDB ID': 'ECK120001204/ECK120001253',...","[{'name': 'leuLABCD attenuator terminator', 'R...",28,0.0,1.0,leuA/leuL,NH4Cl(1),0.0,KH2PO4(3),0.59,{},0.0,"(83590, 83590)",0.0,BOP27,MgSO4(0.24),trace elements,511145,1.0,37 celsius,{ECK120034959},{'ECK120001204/ECK120001253': ['ECK125143560']},"[{'name': 'leuLABCD', 'RegulonDB ID': 'ECK1200...",{'ECK120014726': ['ECK120001204/ECK120001253']},[{'name': 'Amino acid transport and metabolism...,{'Amino acid transport and metabolism': ['ECK1...,"[{'name': 'LeuO', 'mutation set count': 3.0, '...",{'LeuO': ['leuLABCD']},"[{'name': 'Valine, leucine and isoleucine bios...","{'Valine, leucine and isoleucine biosynthesis'..."
1815,intergenic (+174/+54),573114.0,SNP,543207.0,{},NC_000913,glxK/allE,A→T,{},{},25.0,{},M9,glycerol(2.5),{},0.0,0.0,GYD,36.0,{},1.0,0.0,"[{'RegulonDB ID': 'ECK120002891/ECK120002892',...","[{'RegulonDB ID': 'ECK120002891/ECK120002892',...",100,0.0,1.0,glxK/allE,NH4Cl(1),0.0,KH2PO4(3),0.50,{},0.0,"(543207, 543207)",0.0,BOP27,MgSO4(0.24),trace elements,511145,2.0,37 celsius,{},{'ECK120002891/ECK120002892': ['ECK120002891/E...,[],{},[],{},[],{},[],{}
1816,intergenic (+177/+51),573115.0,SNP,543210.0,{},NC_000913,glxK/allE,A→T,{},{},25.0,{},M9,glycerol(2.5),{},0.0,0.0,GYD,36.0,{},1.0,0.0,"[{'RegulonDB ID': 'ECK120002891/ECK120002892',...","[{'RegulonDB ID': 'ECK120002891/ECK120002892',...",101,0.0,1.0,glxK/allE,NH4Cl(1),0.0,KH2PO4(3),0.50,{},0.0,"(543210, 543210)",0.0,BOP27,MgSO4(0.24),trace elements,511145,2.0,37 celsius,{},{'ECK120002891/ECK120002892': ['ECK120002891/E...,[],{},[],{},[],

In [6]:
col_l = [
    "Mutation Type",
    "disrupt",
    "repaired",
    "genomic features",
    "genetic features",
    "genetic feature links",
    "operons",
    "operon links",
    "regulators",
    "regulator links",
    "COGs",
    "COG links",
    "pathways",
    "pathway links",
    "RegulonDB Gene",  # for debugging
    "exp"
]

annot_df = muts_df.copy()
annot_df = annot_df[col_l]
# annot_df["mutation effect"] = annot_df.apply(lambda mut_row:"repaired" if mut_row["repaired"] else ("disrupt" if mut_row["disrupt"] else "unknown"), axis=1)


display(annot_df.shape, annot_df.head())

(107, 16)

,Mutation Type,disrupt,repaired,genomic features,genetic features,genetic feature links,operons,operon links,regulators,regulator links,COGs,COG links,pathways,pathway links,RegulonDB Gene,exp
1737,SNP,0.0,0.0,"[{'name': 'leuLABCD attenuator terminator', 'R...","[{'RegulonDB ID': 'ECK120001204/ECK120001253',...",{'ECK120001204/ECK120001253': ['ECK125143560']},"[{'name': 'leuLABCD', 'RegulonDB ID': 'ECK1200...",{'ECK120014726': ['ECK120001204/ECK120001253']},"[{'name': 'LeuO', 'mutation set count': 3.0, '...",{'LeuO': ['leuLABCD']},[{'name': 'Amino acid transport and metabolism...,{'Amino acid transport and metabolism': ['ECK1...,"[{'name': 'Valine, leucine and isoleucine bios...","{'Valine, leucine and isoleucine biosynthesis'...",leuA/leuL,GYD
1763,SNP,0.0,0.0,"[{'name': 'leuLABCD attenuator terminator', 'R...","[{'RegulonDB ID': 'ECK120001204/ECK120001253',...",{'ECK120001204/ECK120001253': ['ECK125143560']},"[{'name': 'leuLABCD', 'RegulonDB ID': 'ECK1200...",{'ECK120014726': ['ECK120001204/ECK120001253']},"[{'name': 'LeuO', 'mutation set count': 3.0, '...",{'LeuO': ['leuLABCD']},[{'name': 'Amino acid transport and metabolism...,{'Amino acid transport and metabolism': ['ECK1...,"[{'name': 'Valine, leucine and isoleucine bios...","{'Valine, leucine and isoleucine biosynthesis'...",leuA/leuL,GYD
1738,SNP,0.0,0.0,"[{'name': 'leuLABCD attenuator terminator', 'R...","[{'RegulonDB ID': 'ECK120001204/ECK120001253',...",{'ECK120001204/ECK120001253': ['ECK125143560']},"[{'name': 'leuLABCD', 'RegulonDB ID': 'ECK1200...",{'ECK120014726': ['ECK120001204/ECK120001253']},"[{'name': 'LeuO', 'mutation set count': 3.0, '...",{'LeuO': ['leuLABCD']},[{'name': 'Amino acid transport and metabolism...,{'Amino acid transport and metabolism': ['ECK1...,"[{'name': 'Valine, leucine and isoleucine bios...","{'Valine, leucine and isoleucine biosynthesis'...",leuA/leuL,GYD
1815,SNP,0.0,0.0,"[{'RegulonDB ID': 'ECK120002891/ECK120002892',...","[{'RegulonDB ID': 'ECK120002891/ECK120002892',...",{'ECK120002891/ECK120002892': ['ECK120002891/E...,[],{},[],{},[],{},[],{},glxK/allE,GYD
1816,SNP,0.0,0.0,"[{'RegulonDB ID': 'ECK120002891/ECK120002892',...","[{'RegulonDB ID': 'ECK120002891/ECK120002892',...",{'ECK120002891/ECK120002892': ['ECK120002891/E...,[],{},[],{},[],{},[],{},glxK/allE,GYD


In [7]:
# Just removing all "Function unknown" links since connections to this COG isn't useful.
for i, r in annot_df.iterrows():
    if "Function unknown" in r["COG links"].keys():
        del r["COG links"]["Function unknown"]

In [8]:
mut_type_feat_effect_counts = dict()
genomic_feat_counts = dict()
genetic_feat_counts = dict()
operon_counts = dict()
COG_counts = dict()
regulator_counts = dict()
pathway_counts = dict()
# Using the individual feature columns rather than link columns because this guarantees considering all linked and non-linked features
# for each of these levels, except genomic features.
for _, r in annot_df.iterrows():
    
    for genomic_feat in r["genomic features"]:
        
        # These are usually exclusive of each other per feature.
        # There is a design problem here: A multi-feature mutation could disrupt feature (gene) and repair another. Need to measure the predicted effect per feature rather than mutation to enable this (considering a design bug fix).
        effect = ""
        if r["disrupt"]: effect = "disrupt"
        if r["repaired"]: effect = "repair"
            
        mut_type_feat_effect = genomic_feat["name"] + ' ' + r["Mutation Type"] + ' ' + effect
        if mut_type_feat_effect not in mut_type_feat_effect_counts.keys():
            mut_type_feat_effect_counts[mut_type_feat_effect] = 0
        mut_type_feat_effect_counts[mut_type_feat_effect] += 1
    
    for genomic_feat in r["genomic features"]:
        genomic_feat_annot = genomic_feat["RegulonDB ID"] #+ ' ' + r["Mutation Type"]
        if genomic_feat_annot not in genomic_feat_counts.keys():
            genomic_feat_counts[genomic_feat_annot] = 0
        genomic_feat_counts[genomic_feat_annot] += 1

    for genetic_feat, links in r["genetic feature links"].items():
        if genetic_feat not in genetic_feat_counts.keys():
            genetic_feat_counts[genetic_feat] = 0
        genetic_feat_counts[genetic_feat] += len(links)

    # count the operons according to the amount of links from genes since an operon is only included if its gene is hit.
    for operon, links in r["operon links"].items():
        if operon not in operon_counts.keys():
            operon_counts[operon] = 0
        operon_counts[operon] += len(links)

    # count the COGs according to the amount of links from operons since a COG is only included if linked to a mutated operon (its genes)
    for COG, links in r["COG links"].items():
        if COG not in COG_counts.keys():
            COG_counts[COG] = 0
        COG_counts[COG] += len(links)
        
    for reg, links in r["regulator links"].items():
        if reg not in regulator_counts.keys():
            regulator_counts[reg] = 0
        regulator_counts[reg] += len(links)
        
    for pathway, links in r["pathway links"].items():
        if pathway not in pathway_counts.keys():
            pathway_counts[pathway] = 0
        pathway_counts[pathway] += len(links)

In [9]:
operon_df = pd.read_csv("./data/RegulonDB10/operon.txt", sep="\t", comment='#', header=None)
operon_df.columns = [
    "OPERON_ID",
    "OPERON_NAME",
    "FIRSTGENEPOSLEFT",
    "LASTGENEPOSRIGHT",
    "REGULATIONPOSLEFT",
    "REGULATIONPOSRIGHT",
    "OPERON_STRAND",
    "OPERON_INTERNAL_COMMENT",
    "KEY_ID_ORG"]
operon_df["range"] = operon_df.apply(lambda row: (row["REGULATIONPOSLEFT"], row["REGULATIONPOSRIGHT"]), axis=1)

In [10]:
# operon names can be the same as a gene, which is problemmatic for floweaver
def _adjust_name_for_floweaver(operon):
    return operon + ' '


def _get_feat_d(json, RegulonDB_ID=None, name=None):
    feat_d = dict()
    for d in json:
        if (RegulonDB_ID and d["RegulonDB ID"] == RegulonDB_ID) or (name and d["name"] == name):
            feat_d = d
            break
    return feat_d


# Will probably have to put elsewhere at some point due to annotation work (count, significance)
mut_feat_effects = list()
genomic_feats = list()
genetic_feats = list()
operons = list()
COGs = list()
regulators = list()
pathways = list()


flow_df = pd.DataFrame()
for _, r in annot_df.iterrows():

    for genomic_feat_d in r["genomic features"]:
        
        effect = ""
        if r["disrupt"]: effect = "disrupt"
        if r["repaired"]: effect = "repair"    
        mut_type_feat_effect = genomic_feat_d["name"] + ' ' + r["Mutation Type"] + ' ' + effect
        mut_type_feat_effect = str(mut_type_feat_effect_counts[mut_type_feat_effect]) + ' ' + mut_type_feat_effect
        mut_feat_effects.append(mut_type_feat_effect)
            
        genomic_feat_annot = ""
        mut_count = genomic_feat_counts[genomic_feat_d["RegulonDB ID"]]
        if genomic_feat_d["significant"] and mut_count > 1: genomic_feat_annot += "*"
        genomic_feat_annot += str(mut_count) + ' '
        genomic_feat_annot += str(genomic_feat_d["name"])
        flow_row = {
            "source": genomic_feat_annot,
            "target": mut_type_feat_effect,
#                 "type": r["mutation effect"],
            "type": r["exp"],
#             "type": "unknown",
            "value": 1
        }
        flow_df = flow_df.append(flow_row, ignore_index=True)
        
    
    
    for genetic_feat_ID, genomic_feat_set in r["genetic feature links"].items():
        genetic_feat_d = _get_feat_d(RegulonDB_ID=genetic_feat_ID, json=r["genetic features"])
        genetic_feat_annot = ""
        mut_count = genetic_feat_counts[genetic_feat_ID]
        if genetic_feat_d["significant"] and mut_count > 1: genetic_feat_annot += "*"
        genetic_feat_annot += str(mut_count) + ' '
        genetic_feat_annot += str(genetic_feat_d["name"])
        genetic_feats.append(genetic_feat_annot)
        for genomic_feat_ID in genomic_feat_set:
            genomic_feat_d = _get_feat_d(RegulonDB_ID=genomic_feat_ID, json=r["genomic features"])
            k = genomic_feat_ID #+ ' ' + r["Mutation Type"]
            genomic_feat_annot = ""
            mut_count = genomic_feat_counts[k]
            if genomic_feat_d["significant"] and mut_count > 1: genomic_feat_annot += "*"
            genomic_feat_annot += str(mut_count) + ' '
            genomic_feat_annot += str(genomic_feat_d["name"]) #+ ' ' + r["Mutation Type"]
#             if r["disrupt"]: genomic_feat_annot += " disrupt"
#             if r["repaired"]: genomic_feat_annot += " repair"
            genomic_feats.append(genomic_feat_annot)
            flow_row = {
                "source": genetic_feat_annot,
                "target": genomic_feat_annot,
#                 "type": r["mutation effect"],
                "type": r["exp"],
#                 "type": "unknown",
                "value": 1
            }
            flow_df = flow_df.append(flow_row, ignore_index=True)
    
    
    for operon_id, gen_targs in r["operon links"].items():
        operon_d = _get_feat_d(RegulonDB_ID=operon_id, json=r["operons"])
        operon_name = _adjust_name_for_floweaver(operon_d["name"])
        operon_annot = ""
        mut_count = operon_counts[operon_id]
        if operon_d["significant"] and mut_count > 1: operon_annot += "*"
        operon_annot += str(mut_count) + ' '
        operon_annot += operon_name
        operons.append(operon_annot)
        for genetic_feat_ID in gen_targs:
            genetic_feat_d = _get_feat_d(RegulonDB_ID=genetic_feat_ID, json=r["genetic features"])
            genetic_feat_annot = ""
            mut_count = genetic_feat_counts[genetic_feat_ID]
            if genetic_feat_d["significant"] and mut_count > 1: genetic_feat_annot += "*"
            genetic_feat_annot += str(mut_count) + ' '
            genetic_feat_annot += str(genetic_feat_d["name"])
            genetic_feats.append(genetic_feat_annot)
            flow_row = {
                "source": operon_annot,
                "target": genetic_feat_annot,
#                 "type": r["mutation effect"],
                "type": r["exp"],
#                 "type": "unknown",
                "value": 1
            }
            flow_df = flow_df.append(flow_row, ignore_index=True)
            
            
    if FUNC_ANNOT == "pathway":
        for pathway, ops in r["pathway links"].items():
#             if pathway == "Glycerolipid metabolism":  # for GYD Glycerolipid metabolism.
            pathway_feat_d = _get_feat_d(name=pathway, json=r["pathways"])
            pathway_annot = ""
            mut_count = pathway_counts[pathway]
            if pathway_feat_d["significant"] and mut_count > 1: pathway_annot += "*"
            pathway_annot += str(mut_count) + ' ' + pathway_feat_d["name"]
            pathways.append(pathway_annot)
            for operon_id in ops:
                op_feat_d = _get_feat_d(RegulonDB_ID=operon_id, json=r["operons"])
                operon_name = operon_df[operon_df["OPERON_ID"]==operon].iloc[0]["OPERON_NAME"]
                operon_name = _adjust_name_for_floweaver(op_feat_d["name"])
                operon_annot = ""
                mut_count = operon_counts[op_feat_d["RegulonDB ID"]]
                if op_feat_d["significant"] and mut_count > 1: operon_annot += "*"
                operon_annot += str(mut_count) + ' '
                operon_annot += operon_name
                operons.append(operon_annot)
                flow_row = {
                    "source": pathway_annot,
                    "target": operon_annot,
#                     "type": r["mutation effect"],
                    "type": r["exp"],
#                     "type": "unknown",
                    "value": 1
                }
                flow_df = flow_df.append(flow_row, ignore_index=True)
            
            
    if FUNC_ANNOT == "regulon":
        for regulator, ops in r["regulator links"].items():
#             if regulator == "CRP":  # for GYD CRP fig
#             if regulator == "GlpR":  # for GYD GlpR fig
#             if regulator == "Mlc":  # for GYD Mlc fig
            reg_feat_d = _get_feat_d(name=regulator, json=r["regulators"])
            reg_annot = ""
            mut_count = regulator_counts[regulator]
            if reg_feat_d["significant"] and mut_count > 1: reg_annot += "*"
            reg_annot += str(mut_count) + ' ' + reg_feat_d["name"]
            regulators.append(reg_annot)
            for operon_name in ops:
                op_feat_d = _get_feat_d(name=operon_name, json=r["operons"])
                operon_name = _adjust_name_for_floweaver(op_feat_d["name"])
                operon_annot = ""
                mut_count = operon_counts[op_feat_d["RegulonDB ID"]]
                if op_feat_d["significant"] and mut_count > 1: operon_annot += "*"
                operon_annot += str(mut_count) + ' '
                operon_annot += operon_name
                operons.append(operon_annot)
                flow_row = {
                    "source": reg_annot,
                    "target": operon_annot,
#                     "type": r["mutation effect"],
                    "type": r["exp"],
#                     "type": "unknown",
                    "value": 1
                }
                flow_df = flow_df.append(flow_row, ignore_index=True)

                
    if FUNC_ANNOT == "COG":
        for COG, ops in r["COG links"].items():
#             if COG == "Nucleotide transport and metabolism":
            COG_feat_d = _get_feat_d(name=COG, json=r["COGs"])
            COG_annot = ""
            mut_count = COG_counts[COG]
            if COG_feat_d["significant"] and mut_count > 1: COG_annot += "*"
            COG_annot += str(mut_count) + ' ' + COG_feat_d["name"]
            COGs.append(COG_annot)
            for operon_id in ops:
                op_feat_d = _get_feat_d(RegulonDB_ID=operon_id, json=r["operons"])
                operon_name = operon_df[operon_df["OPERON_ID"]==operon].iloc[0]["OPERON_NAME"]
                operon_name = _adjust_name_for_floweaver(op_feat_d["name"])
                operon_annot = ""
                mut_count = operon_counts[op_feat_d["RegulonDB ID"]]
                if op_feat_d["significant"] and mut_count > 1: operon_annot += "*"
                operon_annot += str(mut_count) + ' '
                operon_annot += operon_name
                operons.append(operon_annot)
                flow_row = {
                    "source": COG_annot,
                    "target": operon_annot,
#                     "type": r["mutation effect"],
                    "type": r["exp"],
#                     "type": "unknown",
                    "value": 1
                }
                flow_df = flow_df.append(flow_row, ignore_index=True)

    
display(flow_df[:20])

,source,target,type,value
0,*3 leuLABCD attenuator terminator,3 leuLABCD attenuator terminator SNP,GYD,1.0
1,*3 leuA/leuL,*3 leuLABCD attenuator terminator,GYD,1.0
2,*3 leuLABCD,*3 leuA/leuL,GYD,1.0
3,3 LeuO,*3 leuLABCD,GYD,1.0
4,*3 leuLABCD attenuator terminator,3 leuLABCD attenuator terminator SNP,GYD,1.0
5,*3 leuA/leuL,*3 leuLABCD attenuator terminator,GYD,1.0
6,*3 leuLABCD,*3 leuA/leuL,GYD,1.0
7,3 LeuO,*3 leuLABCD,GYD,1.0
8,*3 leuLABCD attenuator terminator,3 leuLABCD attenuator terminator SNP,GYD,1.0
9,*3 leuA/leuL,*3 leuLABCD attenuator terminator,GYD,1.0


In [11]:
# processing to help a cleaner order of features.
# remove duplicates but maintains the order established above
def remove_duplicates(feats):
    l = list()
    for x in feats:
        if x not in l:
            l.append(x)
    return l

mut_feat_effects = remove_duplicates(mut_feat_effects)
genomic_feats = remove_duplicates(genomic_feats)
genetic_feats = remove_duplicates(genetic_feats)
operons = remove_duplicates(operons)
pathways = remove_duplicates(pathways)
regulators = remove_duplicates(regulators)
COGs = remove_duplicates(COGs)

In [12]:
from ipysankeywidget import SankeyWidget
from floweaver import *


nodes = {
    "mutation": ProcessGroup(mut_feat_effects, title="mutation"),
    "genomic feature": ProcessGroup(genomic_feats, title="genomic feature"),
    "genetic feature": ProcessGroup(genetic_feats, title="genetic feature"),
    "operon": ProcessGroup(operons, title="operon"),
}

l = []
if FUNC_ANNOT == "pathway":
    l = pathways
if FUNC_ANNOT == "regulon":
    l = regulators
if FUNC_ANNOT == "COG":
    l = COGs
nodes[FUNC_ANNOT] = ProcessGroup(l, title=FUNC_ANNOT)


# !!! These partition DFs need to have accessible variables (ex: mut_type_partition) accessible external to the nodes
# otherwise floweaver will crash during its rendering step.
mutation_partition = Partition.Simple("process", mut_feat_effects)
nodes["mutation"].partition = mutation_partition

genomic_feat_partition = Partition.Simple("process", genomic_feats)
nodes["genomic feature"].partition = genomic_feat_partition

genetic_feat_partition = Partition.Simple("process", genetic_feats)
nodes["genetic feature"].partition = genetic_feat_partition

operon_partition = Partition.Simple("process", operons)
nodes["operon"].partition = operon_partition

nodes[FUNC_ANNOT].partition = Partition.Simple("process", l)


# The order that the labels is entered into the bundles is explicit. Don't arbitrarily give an order.
bundles = [
    Bundle(
        "genomic feature",
        "mutation"
    ),
    Bundle(
        "genetic feature",
        "genomic feature",
    ),
    Bundle(
        "operon",
        "genetic feature",
    ),
    Bundle(
        FUNC_ANNOT,
        "operon",
    ),
]


ordering = [
    [FUNC_ANNOT],
    ["operon"],
    ["genetic feature"],
    ["genomic feature"],
    ["mutation"]
]

#
# mut_effect_partition = Partition.Simple(
#     "type", ["unknown", "disrupt", "repaired"])
exp_flow_partition = Partition.Simple(
    "type", ["GYD", "SSW_GLY", "repaired"])
sdd = SankeyDefinition(nodes,
                       bundles,
                       ordering,
                       flow_partition=exp_flow_partition
                      )


# palette_d = {"disrupt": "#C44E52", "unknown": "#6BAAD3", "repaired": "#55A868"}
palette_d = {"GYD": "#6BAAD3", "SSW_GLY": "#DD8452"}
size = dict(width=1400, height=900)
results = weave(sdd,
                flow_df,
                palette=palette_d,
                ).to_widget(
    margins=dict(left=400, right=350),
    **size)
results.auto_save_svg("mutflow.svg")
results.auto_save_png("mutflow.png")
results

SankeyWidget(groups=[{'id': 'regulon', 'type': 'process', 'title': 'regulon', 'nodes': ['regulon^3 LeuO', 'reg…